In [282]:
import pandas as pd
import numpy as np
import requests
import json
import time

api_key = 'RGAPI-6be3f8a6-6833-4e82-a374-00aba6f803fd'

champions = requests.get('http://ddragon.leagueoflegends.com/cdn/11.9.1/data/pt_BR/champion.json').json()
champions_df = np.transpose(pd.DataFrame(champions['data']))
champions_ids = champions_df.loc[:, ['id','key']].rename(columns={'key':'championId', 'id':'championName'})
champions_ids['championId'] = champions_ids.championId.astype('int64')

In [286]:
def get_match_infos_from_id(matchId, api_key, version=4):
    if(version == 4):
        url = f'https://br1.api.riotgames.com/lol/match/v4/matches/{matchId}'
    elif(version == 5):
        url = f'https://americas.api.riotgames.com/lol/match/v5/matches/BR1_{matchId}'
    header = {'api_key':api_key}
    r = requests.get(url, params=header)
    print(f'Partida: {matchId}. Status code: {r.status_code}')
    return r.json()

def get_id_from_url(matchUrl):
    match_details = matchUrl.split('#match-details/')[1]
    return match_details.split('/')[1]

def get_match_infos_from_url(matchUrl, api_key, version=4):
    matchId = get_id_from_url(matchUrl)
    return get_match_infos_from_id(matchId, api_key, version=version)

def get_list_of_ids_from_ulrs(list_of_urls):
    list_of_ids = []
    for url in list_of_urls:
        list_of_ids.append(get_id_from_url(url))
    return np.unique(list_of_ids)

def get_multiple_infos_from_urls(list_of_urls, api_key, version=4):
    list_of_infos = []
    for index, url in enumerate(np.unique(list_of_urls)):
        list_of_infos.append(get_match_infos_from_url(url, api_key, version=version))
        if index % 19 == 0:
            time.sleep(1)
    return list_of_infos

def info_to_players_dataframe(match_infos, version=4):
    if(version == 4):
        players = pd.DataFrame(match_infos['participants'])
        players = pd.concat([players, players.stats.apply(pd.Series)], axis=1)
        players.drop('stats', axis='columns', inplace=True)
        players = players.merge(champions_ids, how='left', on='championId')

        players['gameId']             = match_infos['gameId']
        players['gameDuration']       = match_infos['gameDuration']
        # players['gameStartTimestamp'] = match_infos['gameStartTimestamp']

    elif(version == 5):
        players = pd.DataFrame(match_infos['info']['participants'])
        players['gameId']             = match_infos['info']['gameId']
        players['gameDuration']       = match_infos['info']['gameDuration']
        players['gameStartTimestamp'] = match_infos['info']['gameStartTimestamp']

    return players

def csv_urls_to_participants_df(path_to_csv, api_key, version=4):
    full_match_list = list(pd.read_csv(path_to_csv).match_url)

    list_of_infos  = get_multiple_infos_from_urls(full_match_list, api_key, version=version)
    matches_as_dfs = [info_to_players_dataframe(info, version=version) for info in list_of_infos]
    full_matches   = pd.concat(matches_as_dfs)

    return full_matches

In [287]:
df_full = csv_urls_to_participants_df('../list_of_matches.csv', api_key, version=4)

Partida: 2254123794. Status code: 200


In [290]:
for col in df_full.columns:
    print(col)

participantId
teamId
championId
spell1Id
spell2Id
timeline
participantId
win
item0
item1
item2
item3
item4
item5
item6
kills
deaths
assists
largestKillingSpree
largestMultiKill
killingSprees
longestTimeSpentLiving
doubleKills
tripleKills
quadraKills
pentaKills
unrealKills
totalDamageDealt
magicDamageDealt
physicalDamageDealt
trueDamageDealt
largestCriticalStrike
totalDamageDealtToChampions
magicDamageDealtToChampions
physicalDamageDealtToChampions
trueDamageDealtToChampions
totalHeal
totalUnitsHealed
damageSelfMitigated
damageDealtToObjectives
damageDealtToTurrets
visionScore
timeCCingOthers
totalDamageTaken
magicalDamageTaken
physicalDamageTaken
trueDamageTaken
goldEarned
goldSpent
turretKills
inhibitorKills
totalMinionsKilled
neutralMinionsKilled
neutralMinionsKilledTeamJungle
neutralMinionsKilledEnemyJungle
totalTimeCrowdControlDealt
champLevel
visionWardsBoughtInGame
sightWardsBoughtInGame
wardsPlaced
wardsKilled
firstBloodKill
firstBloodAssist
firstTowerKill
firstTowerAssist
first

In [223]:
infos_v5 = get_match_infos_from_id(2254123794, api_key, version=5)
df_v5    = info_to_players_dataframe(infos_v5, version=5)

Partida: 2254123794. Status code: 200


In [291]:
df_full_v5 = csv_urls_to_participants_df('../list_of_matches.csv', api_key, version=5)

Partida: 2254123794. Status code: 200


In [294]:
cols_analise = ['puuid', 'summonerName','win','detectorWardsPlaced', 'gameId']
df_full_v5.loc[:, cols_analise]

,puuid,summonerName,win,detectorWardsPlaced,gameId
0,q2vqZN9CCembJV_C2aIpaT8qSGmyW-H4cA92g7y4mWH4QO...,Ohto,True,4,2254123794
1,Xl9wzyX9Hnw6v3lFKpSvISadkxJO2KyS4AOs8lxirULLTL...,Axium,True,1,2254123794
2,ouK21DUGdQa1S-38CPYKPsZVQzd5o72Uk4gn_FWzK7Biku...,kindle eyes,True,4,2254123794
3,dSrm4Xq9PEcPGX2mFJ5NfDe0l4IlttTLGvk42VBtS7bNkl...,Nabruxa,True,1,2254123794
4,w42D9MWwWYgP6WBnePIVvA3odKnIYB-LvmjH2pvN6MKUln...,LadyKell,True,2,2254123794
5,Vctog9y_7tyGkSVwiSrgeLmpIs5_5r_BEetZ8888mL3oRf...,Jpguima,False,0,2254123794
6,U-l5FEkys6K6fVlYDHCLT7kzi3A2x015vjpGCC80Vi5SQ5...,EduTenorio,False,1,2254123794
7,EceUuasZqIXn5mPpnnlQvyMd-oKtbl03Q0RdkERw3iCFf2...,coutos simiae,False,3,2254123794
8,MaAGkvfuraI5CSUjbyLptqYi-Enw5Cp-rWRCtSKT8JTt8M...,BobaDeathFett,False,4,2254123794
9,pPAANYjxgtvO8YPiSSjT6j9qHRbPfdE6ZNLfT6Ndyxk-pm...,vitors simiae,False,3,2254123794


In [78]:
personal_url = 'https://matchhistory.br.leagueoflegends.com/pt/#match-details/BR1/2254193087/202978686?tab=overview'